<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/GPT2_Chatbot_Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Jul  9 15:37:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install tqdm
!pip install kobert-transformers==0.4.1
!pip install kogpt2-transformers==0.3.0
!pip install transformers==3.0.2
!pip install torch
!pip install tokenizers==0.8.1rc1
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# 구글 드라이브 연결
import os
import numpy as np
from tqdm import tqdm
import logging
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/HelloNewWorld/gpt/"

import torch
import torch.nn as nn
from torch.utils.data import dataloader, Dataset
from kogpt2_transformers import get_kogpt2_tokenizer, get_kogpt2_model

from transformers import glue_compute_metrics as compute_metrics
from transformers import glue_output_modes as output_modes
from transformers import glue_processors as processors
from transformers import glue_convert_examples_to_features as convert_examples_to_features

import torch.nn.utils.prune as prune

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [4]:
# 로깅 준비
logger = logging.getLogger(__name__)
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.WARN)

logging.getLogger("transformers.modeling_utils").setLevel(
                    logging.WARN)  # 로깅 줄이기

print(torch.__version__)

1.11.0+cu113


In [5]:
torch.set_num_threads(1)
print(torch.__config__.parallel_info())

ATen/Parallel:
	at::get_num_threads() : 1
	at::get_num_interop_threads() : 2
OpenMP 201511 (a.k.a. OpenMP 4.5)
	omp_get_max_threads() : 1
Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
	mkl_get_max_threads() : 1
Intel(R) MKL-DNN v2.5.2 (Git Hash a9302535553c73243c632ad3c4c80beec3d19a1e)
std::thread::hardware_concurrency() : 4
Environment variables:
	OMP_NUM_THREADS : [not set]
	MKL_NUM_THREADS : [not set]
ATen parallel backend: OpenMP



In [6]:
# !python3 '/content/gdrive/My Drive/Colab Notebooks/HelloNewWorld/gpt/preprocess/training_data.py'

In [7]:
class DialogKoGPT2(nn.Module):
  def __init__(self):
    super(DialogKoGPT2, self).__init__()
    self.kogpt2 = get_kogpt2_model()

  def generate(self,
               input_ids,
               do_sample=True,
               max_length= 40,
               top_p=0.92,
               top_k=50,
               temperature= 0.6,
               no_repeat_ngram_size =None,
               num_return_sequences=3,
               early_stopping=False,
               ):
    return self.kogpt2.generate(input_ids,
               do_sample=do_sample,
               max_length=max_length,
               top_p = top_p,
               top_k=top_k,
               temperature=temperature,
               no_repeat_ngram_size= no_repeat_ngram_size,
               num_return_sequences=num_return_sequences,
               early_stopping = early_stopping,
              )

  def forward(self, input, labels = None):
    if labels is not None:
      outputs = self.kogpt2(input, labels=labels)
    else:
      outputs = self.kogpt2(input)

    return outputs

In [8]:
class WellnessAutoRegressiveDataset(Dataset):
  """Wellness Auto Regressive Dataset"""

  def __init__(self,
               file_path = path+"data/wellness_dialog_for_autoregressive.txt",
               n_ctx = 1024
               ):
    self.file_path = file_path
    self.data =[]
    self.tokenizer = get_kogpt2_tokenizer()


    bos_token_id = [self.tokenizer.bos_token_id]
    eos_token_id = [self.tokenizer.eos_token_id]
    pad_token_id = [self.tokenizer.pad_token_id]

    file = open(self.file_path, 'r', encoding='utf-8')

    while True:
      line = file.readline()
      if not line:
        break
      datas = line.split("\t")
      index_of_words = bos_token_id +self.tokenizer.encode(datas[0]) + eos_token_id + bos_token_id + self.tokenizer.encode(datas[1][:-1])+ eos_token_id
      pad_token_len = n_ctx - len(index_of_words)

      index_of_words += pad_token_id * pad_token_len

      self.data.append(index_of_words)

    file.close()

  def __len__(self):
    return len(self.data)

  def __getitem__(self,index):
    item = self.data[index]
    return item

In [9]:
dataset = WellnessAutoRegressiveDataset()

In [10]:
root_path="gdrive/My Drive/Colab Notebooks/HelloNewWorld/gpt"
data_path = f"{root_path}/data/chatbot_wellness_dialog_for_autoregressive.txt"
checkpoint_path =f"{root_path}/checkpoint"
load_ckpt_path = f"{checkpoint_path}/kogpt2-20220710.pth"
save_ckpt_path = f"{checkpoint_path}/kogpt2-20220710-add-chatbotdata.pth"

n_epoch = 3         # Num of Epoch
batch_size = 2      # 배치 사이즈
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)
save_step = 500 # 학습 저장 주기
learning_rate = 5e-5  # Learning Rate

dataset= WellnessAutoRegressiveDataset(data_path)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [11]:
model = DialogKoGPT2()

In [12]:
# 아직 Pruning이 적용되지 않은 모델 구조 출력
model

DialogKoGPT2(
  (kogpt2): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50000, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
         

In [13]:
# Block이 12개 중첩되는 형태이므로 하나만 출력해봄
model.kogpt2.transformer.h[0]

Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [14]:
# Block의 attention에서 Pruning을 진행하므로 attention 모듈만 살펴보기
model.kogpt2.transformer.h[0].attn

Attention(
  (c_attn): Conv1D()
  (c_proj): Conv1D()
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
)

In [15]:
# attention모듈의 c_attn << Pruning을 진행하기 위해 module로 저장
module = model.kogpt2.transformer.h[0].attn.c_attn

In [16]:
# c_attn 파라미터에는 weight, bias가 존재함.
# 현재 named_buffers()에는 아무것도 존재하지 않음.
list(module.named_parameters())

[('weight', Parameter containing:
  tensor([[ 0.3049,  0.1613, -0.2721,  ...,  0.0723, -0.0648, -0.0751],
          [-0.0974, -0.2858, -0.0168,  ...,  0.0236,  0.0090,  0.0111],
          [ 0.2382, -0.7249,  0.1719,  ...,  0.0373, -0.0615,  0.0319],
          ...,
          [-0.4462, -0.3731, -0.5665,  ...,  0.0153, -0.0273, -0.0193],
          [ 0.4190,  0.2665, -0.1696,  ..., -0.0173,  0.0615,  0.0008],
          [-0.3748,  0.0937,  0.5369,  ..., -0.0767, -0.0558, -0.0247]],
         requires_grad=True)), ('bias', Parameter containing:
  tensor([ 0.3300,  0.7072, -0.2997,  ...,  0.0008, -0.0028,  0.0023],
         requires_grad=True))]

In [17]:
# prune 모듈을 활용해 unstructured pruning 적용.
# 임의로 30%의 파라미터에 pruning을 적용함
prune.random_unstructured(module, name="weight", amount=0.3)

Conv1D()

In [18]:
list(module.named_parameters())

[('bias', Parameter containing:
  tensor([ 0.3300,  0.7072, -0.2997,  ...,  0.0008, -0.0028,  0.0023],
         requires_grad=True)), ('weight_orig', Parameter containing:
  tensor([[ 0.3049,  0.1613, -0.2721,  ...,  0.0723, -0.0648, -0.0751],
          [-0.0974, -0.2858, -0.0168,  ...,  0.0236,  0.0090,  0.0111],
          [ 0.2382, -0.7249,  0.1719,  ...,  0.0373, -0.0615,  0.0319],
          ...,
          [-0.4462, -0.3731, -0.5665,  ...,  0.0153, -0.0273, -0.0193],
          [ 0.4190,  0.2665, -0.1696,  ..., -0.0173,  0.0615,  0.0008],
          [-0.3748,  0.0937,  0.5369,  ..., -0.0767, -0.0558, -0.0247]],
         requires_grad=True))]

In [19]:
# Pruning에 의해 생성된 weight_mask
list(module.named_buffers())

[('weight_mask', tensor([[1., 1., 1.,  ..., 1., 1., 0.],
          [1., 1., 1.,  ..., 0., 1., 1.],
          [0., 0., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 0., 1.,  ..., 1., 0., 1.],
          [0., 1., 0.,  ..., 1., 1., 0.],
          [0., 0., 1.,  ..., 0., 0., 1.]]))]

In [20]:
# weight는 더이상 파라미터로 관리되는 것이 아닌 속성으로 관리됨.
module.weight

tensor([[ 0.3049,  0.1613, -0.2721,  ...,  0.0723, -0.0648, -0.0000],
        [-0.0974, -0.2858, -0.0168,  ...,  0.0000,  0.0090,  0.0111],
        [ 0.0000, -0.0000,  0.1719,  ...,  0.0373, -0.0615,  0.0319],
        ...,
        [-0.4462, -0.0000, -0.5665,  ...,  0.0153, -0.0000, -0.0193],
        [ 0.0000,  0.2665, -0.0000,  ..., -0.0173,  0.0615,  0.0000],
        [-0.0000,  0.0000,  0.5369,  ..., -0.0000, -0.0000, -0.0247]],
       grad_fn=<MulBackward0>)

In [21]:
# 앞서 적용한 pruning을 순전파 이전에 적용하기 위해 적용함.
module._forward_pre_hooks

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured at 0x7f37f59b4850>)])

In [22]:
# l1_unstructured를 이용해 bias에도 Pruning을 적용함.
prune.l1_unstructured(module, name='bias', amount=50)

Conv1D()

In [23]:
list(module.named_parameters())

[('weight_orig', Parameter containing:
  tensor([[ 0.3049,  0.1613, -0.2721,  ...,  0.0723, -0.0648, -0.0751],
          [-0.0974, -0.2858, -0.0168,  ...,  0.0236,  0.0090,  0.0111],
          [ 0.2382, -0.7249,  0.1719,  ...,  0.0373, -0.0615,  0.0319],
          ...,
          [-0.4462, -0.3731, -0.5665,  ...,  0.0153, -0.0273, -0.0193],
          [ 0.4190,  0.2665, -0.1696,  ..., -0.0173,  0.0615,  0.0008],
          [-0.3748,  0.0937,  0.5369,  ..., -0.0767, -0.0558, -0.0247]],
         requires_grad=True)), ('bias_orig', Parameter containing:
  tensor([ 0.3300,  0.7072, -0.2997,  ...,  0.0008, -0.0028,  0.0023],
         requires_grad=True))]

In [24]:
list(module.named_buffers())

[('weight_mask', tensor([[1., 1., 1.,  ..., 1., 1., 0.],
          [1., 1., 1.,  ..., 0., 1., 1.],
          [0., 0., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 0., 1.,  ..., 1., 0., 1.],
          [0., 1., 0.,  ..., 1., 1., 0.],
          [0., 0., 1.,  ..., 0., 0., 1.]])),
 ('bias_mask', tensor([1., 1., 1.,  ..., 1., 1., 1.]))]

In [25]:
module.bias

tensor([ 0.3300,  0.7072, -0.2997,  ...,  0.0008, -0.0028,  0.0023],
       grad_fn=<MulBackward0>)

In [26]:
module.bias.size(), (module.bias == 0).sum()

(torch.Size([2304]), tensor(50))

In [27]:
# module의 _forward_pre_hooks에도 2개의 pre_hook이 존재하는 것을 알 수 있음.
# Pruning을 중첩해서 사용 가능함
# 중첩의 의미는, 여러 pruning mask를 적용해서 사용 가능 하단 소리.

module._forward_pre_hooks

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured at 0x7f37f59b4850>),
             (1, <torch.nn.utils.prune.L1Unstructured at 0x7f37f2097b50>)])

In [28]:
model.kogpt2.transformer.h[0]

Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [29]:
# Global Pruning 적용

final_model = DialogKoGPT2()
final_model.to(device)

parameters_to_prune = ()
for i in range(12):
    parameters_to_prune += (
        (final_model.kogpt2.transformer.h[i].attn.c_attn, 'weight'),
        (final_model.kogpt2.transformer.h[i].attn.c_proj, 'weight'),
    )

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

In [30]:
# Global Pruning을 통해 전체 Sparsity를 20%로 만들 수 있음을 알 수 있음.

for i in range(12):
    print(
        "Sparsity in Layer {}-th c_attn weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(final_model.kogpt2.transformer.h[i].attn.c_attn.weight == 0))
            / float(final_model.kogpt2.transformer.h[i].attn.c_attn.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th c_proj weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(final_model.kogpt2.transformer.h[i].attn.c_proj.weight == 0))
            / float(final_model.kogpt2.transformer.h[i].attn.c_proj.weight.nelement())
        )
    )
    print()

    
numerator, denominator = 0, 0
for i in range(12):
    numerator += torch.sum(final_model.kogpt2.transformer.h[i].attn.c_attn.weight == 0)
    numerator += torch.sum(final_model.kogpt2.transformer.h[i].attn.c_proj.weight == 0)

    denominator += final_model.kogpt2.transformer.h[i].attn.c_attn.weight.nelement()
    denominator += final_model.kogpt2.transformer.h[i].attn.c_proj.weight.nelement()
    
print("Global sparsity: {:.2f}%".format(100. * float(numerator) / float(denominator)))

Sparsity in Layer 1-th c_attn weight: 28.47%
Sparsity in Layer 1-th c_proj weight: 65.74%

Sparsity in Layer 2-th c_attn weight: 20.61%
Sparsity in Layer 2-th c_proj weight: 24.94%

Sparsity in Layer 3-th c_attn weight: 18.73%
Sparsity in Layer 3-th c_proj weight: 27.17%

Sparsity in Layer 4-th c_attn weight: 17.68%
Sparsity in Layer 4-th c_proj weight: 23.86%

Sparsity in Layer 5-th c_attn weight: 17.33%
Sparsity in Layer 5-th c_proj weight: 21.64%

Sparsity in Layer 6-th c_attn weight: 17.04%
Sparsity in Layer 6-th c_proj weight: 19.34%

Sparsity in Layer 7-th c_attn weight: 17.24%
Sparsity in Layer 7-th c_proj weight: 18.92%

Sparsity in Layer 8-th c_attn weight: 17.60%
Sparsity in Layer 8-th c_proj weight: 19.29%

Sparsity in Layer 9-th c_attn weight: 16.98%
Sparsity in Layer 9-th c_proj weight: 19.14%

Sparsity in Layer 10-th c_attn weight: 17.09%
Sparsity in Layer 10-th c_proj weight: 17.16%

Sparsity in Layer 11-th c_attn weight: 17.65%
Sparsity in Layer 11-th c_proj weight: 15.

In [31]:
loss_fct = torch.nn.CrossEntropyLoss(ignore_index=3)
optimizer = torch.optim.Adam(final_model.parameters(), lr=learning_rate)

losses =[]
for epoch in range(n_epoch):
    count = 0
    with tqdm(total=len(train_loader), desc=f"Train({epoch})") as pbar:
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            data = torch.stack(data)  # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.
            data = data.transpose(1, 0)
            data= data.to(ctx)

            outputs = final_model(data, labels=data)
            _, logits = outputs[:2]

            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = data[..., 1:].contiguous()

            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

            # if count % 10 == 0:
            #     print('epoch no.{} train no.{}  loss = {}'.format(epoch, count + 1, loss))
            if (count > 0 and count % save_step == 0) or (len(data) < batch_size):
                torch.save({
                    'epoch': epoch,
                    'train_no': count,
                    'model_state_dict': final_model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(), 
                    'loss': loss
                }, save_ckpt_path)
            count += 1
            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")

Train(2): 100%|██████████| 13840/13840 [1:33:31<00:00,  2.47it/s, Loss: 0.853 (1.602)]


In [32]:
torch.save({
    'epoch': epoch,
    'train_no': count,
    'model_state_dict': final_model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(), 
    'loss': loss
}, save_ckpt_path)